In [1]:
# Enviroment setting up
!pip install pandas dash
!pip install jupyter-dash

     |████████████████████████████████| 9.8 MB 26.5 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 13.9 MB/s  eta 0:00:01
     |████████████████████████████████| 357 kB 85.4 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11448 sha256=ea40f3d66947ecafb03aef51f30b52e203f73b5ddfed64fcfa468c2a6b539a29
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/ce/18/7f/e9527e3e66db1456194ac7f61eb3211068c409edceecff2d31
Successfully built retrying


In [2]:
# Data load
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2022-06-19 09:08:07--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2022-06-19 09:08:08 (110 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]



In [3]:
# Import required libraries
from jupyter_dash import JupyterDash
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                {'label': 'KSC LC-39A', 'value':'KSC LC-39A'},
                                                {'label': 'VAFB SLC-4E', 'value':'VAFB SLC-4E'}
                                             ],
                                             # Default dropdown value
                                             value='ALL',
                                             placeholder="Select a launch site",
                                             searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def render_pie_chart(entered_site):
  # Filtered dataframe for else condition
    filtered_df = spacex_df[spacex_df['Launch Site']==str(entered_site)]
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Launch in all sites')
        return fig
    else:
        fig = px.pie(filtered_df, names='class', title=f'Launch in {entered_site}')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id='payload-slider', component_property='value')])

def render_scatter_plot(entered_site, slider_payload):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= slider_payload[0]) & (spacex_df['Payload Mass (kg)'] <= slider_payload[1])]
    if entered_site == 'ALL':
      fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='All site payload and outcome')
      return fig
    else:
      filtered_df = filtered_df[filtered_df['Launch Site']==entered_site]
      fig = px.scatter(filtered_df, x='Payload Mass (kg)', y = 'class', color='Booster Version Category', title=f'Site {entered_site} Payload and outcome')
      return fig

# Run the app
app.run_server(mode='inline', debug=True)